In [1]:
import intake
import os

In [2]:
url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
# open the catalog
dataframe = intake.open_esm_datastore(url)
dataframe.df.columns
df = dataframe.df

/Users/doan/anaconda3/envs/intake_esm/lib/python3.12/site-packages/fastprogress/fastprogress.py:107: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


In [3]:
import re
def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)

In [11]:
sid = 'EC-Earth3'
exp = 'historical'
var = 'tas'
table_id = '6hrPlev'
models = dataframe.search(experiment_id=exp,
                          table_id='Amon',
                          variable_id=var,
                          source_id = sid,
                          #institution_id=ins,
                          #member_id=mem
                          ) 
ml = natural_sort(models.df.member_id.values)

# get the first one only then seach again
mem = ml[0]
model_s = dataframe.search(experiment_id=exp,
                          table_id='Amon',
                          variable_id=var,
                          source_id = sid,
                          #institution_id=ins,
                          member_id=mem
                          )      

In [12]:
model_s.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,EC-Earth-Consortium,EC-Earth3,historical,r1i1p1f1,Amon,tas,gr,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-E...,NaN,20200310


In [8]:
df.table_id.unique()

array(['Amon', '6hrPlev', '3hr', 'day', 'EmonZ', 'E3hr', '6hrPlevPt',
       'AERmon', 'LImon', 'CFmon', 'Lmon', 'fx', 'SImon', 'Ofx', 'Omon',
       'EdayZ', 'Emon', 'CFday', 'AERday', 'Eday', 'Oyr', 'Eyr', 'Oday',
       'SIday', 'AERmonZ', '6hrLev', 'E1hrClimMon', 'CF3hr', 'AERhr',
       'Odec', 'Oclim', 'Efx', 'Aclim', 'SIclim', 'IfxGre', 'ImonGre',
       'Eclim'], dtype=object)

In [ ]:
for sid in [ 'EC-Earth3', 'MIROC6',  'MRI-ESM2-0', 'ACCESS-CM2', 'IPSL-CM6A-LR', 'MPI-ESM1-2-HR'][:1]:
    for exp in ['historical','ssp585', 'ssp126', 'ssp370','ssp245'][:]:
            for var in ['tas','ta','ua','va','hur','zg','ts'][:]:
                # seach all files with information given above
                models = dataframe.search(experiment_id=exp,
                                          table_id='Amon',
                                          variable_id=var,
                                          source_id = sid,
                                          #institution_id=ins,
                                          #member_id=mem
                                          )  
                # then one might get several files with the same conditions
                # r1: Realization (initial condition run)
                # i1: Initialization method
                # p1: Physical parameters
                # f1: External forcings

                print(var, exp, sid,  len(models.df))

                # if no files exist then print out error
                if len(models.df) == 0: print('*** \n Erorrrr \n')

                # sort the possible files
                ml = natural_sort(models.df.member_id.values)

                # get the first one only then seach again
                mem = ml[0]
                model_s = dataframe.search(experiment_id=exp,
                                          table_id='Amon',
                                          variable_id=var,
                                          source_id = sid,
                                          #institution_id=ins,
                                          member_id=mem
                                          )                  

                # if no files exist then print out error
                if len(model_s.df) == 0: print('*** \n Erorrrr \n')

                print(mem)

                if len(model_s.df) > 0:
                    print('Download')
                    
                    if True:
                        
                        try:
                        
                            datasets = model_s.to_dataset_dict(zarr_kwargs={'consolidated': True, "decode_times": True, "use_cftime": True })
                            #datasets = models.to_dataset_dict(xarray_open_kwargs={"consolidated": True, "decode_times": True, "use_cftime": True})
                            
                            for k, v in datasets.items():
                                odir = 'Download_s8/'+sid+'/'+exp+'/'
                                if not os.path.exists(odir): os.makedirs(odir)
                                ofile = odir + var + '_'+ k + '_'+mem+'.nc'
                                print('write to ',ofile)
                                v.to_netcdf(ofile)
                        except:
                            print('fail')
                            

tas historical EC-Earth3 73
r1i1p1f1
Download

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'
write to  Download_s8/EC-Earth3/historical/tas_CMIP.EC-Earth-Consortium.EC-Earth3.historical.Amon.gr_r1i1p1f1.nc
ta historical EC-Earth3 71
r1i1p1f1
Download

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'
write to  Download_s8/EC-Earth3/historical/ta_CMIP.EC-Earth-Consortium.EC-Earth3.historical.Amon.gr_r1i1p1f1.nc
ua historical EC-Earth3 72
r1i1p1f1
Download

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'
write to  Download_s8/EC-Earth3/historical/ua_CMIP.EC-Earth-Consortium.EC-Earth3.historical.Amon.gr_r1i1p1f1.nc
